In [3]:
import random
from datasets import load_dataset
import json

/Users/steve/miniconda3/envs/mlc-chat-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("OpenAssistant/oasst1")
# get pandas dataframe
oasst_df = dataset['train'].to_pandas()
oasst_df.head()

Generating validation split: 100%|██████████| 4401/4401 [00:00<00:00, 454043.34 examples/s]


,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
0,6ab24d72-0181-4594-a9cd-deaf170242fb,None,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00044308538781479, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,6ab24d72-0181-4594-a9cd-deaf170242fb,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00026396565954200923, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [3...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.03648477792739868, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,None,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
3,343ee2d4-87ae-41fd-a768-bdd65959dc4a,6ab24d72-0181-4594-a9cd-deaf170242fb,49ddcb0d-6588-43bd-858d-19c577f12e7b,2023-02-06T13:37:56.044680+00:00,Monopsony is a market structure in which there...,assistant,en,3,True,False,1.0,False,None,"{'toxicity': 0.0008866374846547842, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_labeling...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
4,18145bf4-37fd-4ac0-80f5-6108b5f2b365,343ee2d4-87ae-41fd-a768-bdd65959dc4a,e10e99a0-38ac-4b07-bf5d-4427696e4e0d,2023-02-06T18:52:51.428543+00:00,How can one fight back when a monospony had be...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.0009362137061543763, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1'], 'count': [1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


In [5]:
oasst_df.lang.unique()

array(['en', 'es', 'de', 'ru', 'ja', 'pt-BR', 'ca', 'fr', 'pl', 'vi',
       'zh', 'hu', 'ko', 'eu', 'it', 'uk-UA', 'id', 'ar', 'fi', 'tr',
       'da', 'th', 'sv', 'cs'], dtype=object)

In [6]:
# apply relevant filters
oasst_df = oasst_df[oasst_df['synthetic'] == False]
oasst_en_df = oasst_df[oasst_df['lang'] == 'en']

In [7]:
conversation_ids = oasst_en_df['message_tree_id'].unique().tolist()

random.seed(42)
n_samples = 2000
conversation_ids_sample = random.sample(conversation_ids, n_samples)

conversation_starters = []

for id in conversation_ids_sample:
  conversation_df = oasst_en_df[oasst_en_df['message_tree_id'] == id]
  user_prompt_df = conversation_df[conversation_df.role == "prompter"]
  # filter out text longer than 1500 characters
  user_prompt_df = user_prompt_df[user_prompt_df['text'].str.len() < 1500]
  if user_prompt_df.shape[0] > 1:
    conversation_starters.append(user_prompt_df['text'].values[0])

In [8]:
N_CONVERSASTIONS = 50
MIN_CONVERSATION_LENGTH = 5

conversations = []

for i in range(N_CONVERSASTIONS):
  delta = random.randint(1, 5)
  convo = random.sample(conversation_starters, MIN_CONVERSATION_LENGTH + delta)

  convo = [interaction for interaction in convo]

  conversations.append(convo)


In [53]:
with open("conversations.json", "w") as f:
  json.dump(conversations, f, indent=4)
